In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1165761,2021-04-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1165762,2021-04-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1165763,2021-04-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1165764,2021-04-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
31271,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31273,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31275,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31277,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31279,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
718684,2021-04-09,Arkansas,Arkansas,5001,2082,36.00,5000,Arkansas,AR,Arkansas,State,3017804
718686,2021-04-10,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
718688,2021-04-11,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
718690,2021-04-12,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1165761,2021-04-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1165762,2021-04-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1165763,2021-04-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1165764,2021-04-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-13') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
448,2021-04-13,Snohomish,Washington,34114,564.00,53061,WA,County,822083,4149.70,68.61
894,2021-04-13,Cook,Illinois,515075,10331.00,17031,IL,County,5150233,10001.00,200.59
1339,2021-04-13,Orange,California,268414,4849.00,6059,CA,County,3175692,8452.14,152.69
1783,2021-04-13,Maricopa,Arizona,529271,9739.00,4013,AZ,County,4485414,11799.82,217.13
2227,2021-04-13,Los Angeles,California,1227256,23505.00,6037,CA,County,10039107,12224.75,234.13
...,...,...,...,...,...,...,...,...,...,...,...
1165157,2021-04-13,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1165340,2021-04-13,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1165492,2021-04-13,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1165640,2021-04-13,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
448,2021-04-13,Snohomish,Washington,34114,564.00,53061,WA,County,822083,4149.70,68.61,68.61,4149.70
894,2021-04-13,Cook,Illinois,515075,10331.00,17031,IL,County,5150233,10001.00,200.59,200.59,10001.00
1339,2021-04-13,Orange,California,268414,4849.00,6059,CA,County,3175692,8452.14,152.69,152.69,8452.14
1783,2021-04-13,Maricopa,Arizona,529271,9739.00,4013,AZ,County,4485414,11799.82,217.13,217.13,11799.82
2227,2021-04-13,Los Angeles,California,1227256,23505.00,6037,CA,County,10039107,12224.75,234.13,234.13,12224.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165157,2021-04-13,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1165340,2021-04-13,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1165492,2021-04-13,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1165640,2021-04-13,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
751455,2021-04-13,Hale,Alabama,2187,75.00,1065,AL,County,14651,14927.31,511.91,511.91,14927.31,1
817875,2021-04-13,Clarke,Alabama,3465,60.00,1025,AL,County,23622,14668.53,254.00,254.00,14668.53,2
588081,2021-04-13,Lowndes,Alabama,1365,53.00,1085,AL,County,9726,14034.55,544.93,544.93,14034.55,3
437646,2021-04-13,Franklin,Alabama,4199,82.00,1059,AL,County,31362,13388.81,261.46,261.46,13388.81,4
521482,2021-04-13,Etowah,Alabama,13665,346.00,1055,AL,County,102268,13361.95,338.33,338.33,13361.95,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105971,2021-04-13,Platte,Wyoming,612,11.00,56031,WY,County,8393,7291.79,131.06,131.06,7291.79,19
717167,2021-04-13,Converse,Wyoming,989,17.00,56009,WY,County,13822,7155.26,122.99,122.99,7155.26,20
891392,2021-04-13,Lincoln,Wyoming,1384,12.00,56023,WY,County,19830,6979.32,60.51,60.51,6979.32,21
991918,2021-04-13,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
588081,2021-04-13,Lowndes,Alabama,1365,53.00,1085,AL,County,9726,14034.55,544.93,544.93,14034.55,3,1
751455,2021-04-13,Hale,Alabama,2187,75.00,1065,AL,County,14651,14927.31,511.91,511.91,14927.31,1,2
250257,2021-04-13,Walker,Alabama,7100,275.00,1127,AL,County,63521,11177.41,432.93,432.93,11177.41,28,3
634158,2021-04-13,Greene,Alabama,910,34.00,1063,AL,County,8111,11219.33,419.18,419.18,11219.33,27,4
519942,2021-04-13,Clay,Alabama,1495,55.00,1027,AL,County,13235,11295.81,415.56,415.56,11295.81,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632625,2021-04-13,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
891392,2021-04-13,Lincoln,Wyoming,1384,12.00,56023,WY,County,19830,6979.32,60.51,60.51,6979.32,21,20
869208,2021-04-13,Uinta,Wyoming,2153,12.00,56041,WY,County,20226,10644.71,59.33,59.33,10644.71,4,21
249084,2021-04-13,Teton,Wyoming,3675,9.00,56039,WY,County,23464,15662.29,38.36,38.36,15662.29,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,18,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,18,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,18,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,18,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4149.70,18,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163024,2021-04-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1163025,2021-04-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1163026,2021-04-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1163027,2021-04-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
748339,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,1.00
748340,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
748341,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
748342,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
748343,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14927.31,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906276,2021-04-09,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
906277,2021-04-10,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
906278,2021-04-11,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00
906279,2021-04-12,Crook,Wyoming,425,12.00,56011,WY,County,7584,5603.90,158.23,158.23,5643.46,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
585725,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,1.00,0.00
585726,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
585727,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
585728,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
585729,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14034.55,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583800,2021-04-09,Albany,Wyoming,4055,11.00,56001,WY,County,38880,10429.53,28.29,28.29,10527.26,23,5,17.00,0.00
583801,2021-04-10,Albany,Wyoming,4055,11.00,56001,WY,County,38880,10429.53,28.29,28.29,10527.26,23,5,0.00,0.00
583802,2021-04-11,Albany,Wyoming,4055,11.00,56001,WY,County,38880,10429.53,28.29,28.29,10527.26,23,5,0.00,0.00
583803,2021-04-12,Albany,Wyoming,4079,11.00,56001,WY,County,38880,10491.26,28.29,28.29,10527.26,23,5,24.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-13') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
298709,2021-04-13,Imperial,California,27851,716.00,6025,CA,County,181215,15369.04,395.11,395.11,15369.04,1,2,10.00,0.00
2227,2021-04-13,Los Angeles,California,1227256,23505.00,6037,CA,County,10039107,12224.75,234.13,234.13,12224.75,2,5,405.00,2.00
148805,2021-04-13,San Bernardino,California,293925,4493.00,6071,CA,County,2180085,13482.27,206.09,206.09,13482.27,3,4,90.00,1.00
522974,2021-04-13,Inyo,California,1390,36.00,6027,CA,County,18039,7705.53,199.57,199.57,7705.53,4,25,0.00,0.00
66573,2021-04-13,Stanislaus,California,60083,1020.00,6099,CA,County,550660,10911.09,185.23,185.23,10911.09,5,9,84.00,2.00
30867,2021-04-13,Riverside,California,296801,4504.00,6065,CA,County,2470546,12013.58,182.31,182.31,12013.58,6,6,166.00,12.00
82530,2021-04-13,Tulare,California,49300,824.00,6107,CA,County,466195,10574.97,176.75,176.75,10574.97,7,10,28.00,0.00
52975,2021-04-13,San Joaquin,California,71178,1334.00,6077,CA,County,762148,9339.13,175.03,175.03,9339.13,8,19,184.00,0.00
30061,2021-04-13,Fresno,California,100277,1633.00,6019,CA,County,999101,10036.72,163.45,163.45,10036.72,9,14,47.00,0.00
473969,2021-04-13,Merced,California,31105,449.00,6047,CA,County,277680,11201.74,161.70,161.70,11201.74,10,8,35.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1105522,2021-04-13,Lassen,California,5666,1.00,6035,CA,County,30573,18532.69,3.27,3.27,18532.69,56,1,6.00,0.00
298709,2021-04-13,Imperial,California,27851,716.00,6025,CA,County,181215,15369.04,395.11,395.11,15369.04,1,2,10.00,0.00
634100,2021-04-13,Kings,California,22723,245.00,6031,CA,County,152940,14857.46,160.19,160.19,14857.46,11,3,15.00,2.00
148805,2021-04-13,San Bernardino,California,293925,4493.00,6071,CA,County,2180085,13482.27,206.09,206.09,13482.27,3,4,90.00,1.00
2227,2021-04-13,Los Angeles,California,1227256,23505.00,6037,CA,County,10039107,12224.75,234.13,234.13,12224.75,2,5,405.00,2.00
30867,2021-04-13,Riverside,California,296801,4504.00,6065,CA,County,2470546,12013.58,182.31,182.31,12013.58,6,6,166.00,12.00
187065,2021-04-13,Kern,California,107434,1306.00,6029,CA,County,900202,11934.43,145.08,145.08,11934.43,14,7,60.00,2.00
473969,2021-04-13,Merced,California,31105,449.00,6047,CA,County,277680,11201.74,161.70,161.70,11201.74,10,8,35.00,1.00
66573,2021-04-13,Stanislaus,California,60083,1020.00,6099,CA,County,550660,10911.09,185.23,185.23,10911.09,5,9,84.00,2.00
82530,2021-04-13,Tulare,California,49300,824.00,6107,CA,County,466195,10574.97,176.75,176.75,10574.97,7,10,28.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
298709,2021-04-13,Imperial,California,27851,716.00,6025,CA,County,181215,15369.04,395.11,395.11,15369.04,1,2,10.00,0.00
2227,2021-04-13,Los Angeles,California,1227256,23505.00,6037,CA,County,10039107,12224.75,234.13,234.13,12224.75,2,5,405.00,2.00
148805,2021-04-13,San Bernardino,California,293925,4493.00,6071,CA,County,2180085,13482.27,206.09,206.09,13482.27,3,4,90.00,1.00
522974,2021-04-13,Inyo,California,1390,36.00,6027,CA,County,18039,7705.53,199.57,199.57,7705.53,4,25,0.00,0.00
66573,2021-04-13,Stanislaus,California,60083,1020.00,6099,CA,County,550660,10911.09,185.23,185.23,10911.09,5,9,84.00,2.00
30867,2021-04-13,Riverside,California,296801,4504.00,6065,CA,County,2470546,12013.58,182.31,182.31,12013.58,6,6,166.00,12.00
82530,2021-04-13,Tulare,California,49300,824.00,6107,CA,County,466195,10574.97,176.75,176.75,10574.97,7,10,28.00,0.00
52975,2021-04-13,San Joaquin,California,71178,1334.00,6077,CA,County,762148,9339.13,175.03,175.03,9339.13,8,19,184.00,0.00
30061,2021-04-13,Fresno,California,100277,1633.00,6019,CA,County,999101,10036.72,163.45,163.45,10036.72,9,14,47.00,0.00
473969,2021-04-13,Merced,California,31105,449.00,6047,CA,County,277680,11201.74,161.70,161.70,11201.74,10,8,35.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,395.11,15369.04,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3912,04/09/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,-1.00,0.00
3913,04/10/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3914,04/11/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00
3915,04/12/21,Lassen,5660,1.00,18513.07,3.27,3.27,18532.69,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,395.11,15369.04,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,395.11,15369.04,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3998,04/09/21,Merced,31009,448.00,11167.17,161.34,161.70,11201.74,10,8,24.00,1.00
3999,04/10/21,Merced,31009,448.00,11167.17,161.34,161.70,11201.74,10,8,0.00,0.00
4000,04/11/21,Merced,31009,448.00,11167.17,161.34,161.70,11201.74,10,8,0.00,0.00
4001,04/12/21,Merced,31070,448.00,11189.14,161.34,161.70,11201.74,10,8,61.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)